# Read .mot motion data (HDF5) using tables

### Download a sample file and build decode_motion (Ubuntu 16.04 binary provided)

In [ ]:
#Download a sample video. Takes a while...
import urllib.request
import zipfile

url = "http://downloads.dvdloc8.com/trailers/divxdigest/serenity_hd_dvd-trailer.zip"
urllib.request.urlretrieve(url, "trailer.zip")

with zipfile.ZipFile("./trailer.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [ ]:
! make

In [ ]:
! ./decode_motion trailer.mp4

# Read .mot motion data (HDF5) using tables

In [ ]:
import numpy as np
np.set_printoptions(threshold=np.nan)

In [ ]:
import tables
import os

filename = os.getcwd() + "/motion_vectors.h5"
f = tables.open_file(filename, mode='r')

In [ ]:
f.root.motion_tensor

In [ ]:
a = f.root.motion_tensor
a.shape

In [ ]:
print(a[0][:,:,0])

# Single frame strength of motion

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
frame = 600
U = a[frame][:,:,0]
V = a[frame][:,:,1]
plt.imshow(V+U)
plt.colorbar()
plt.show()

# Multiframe strength of motion

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
 
#plt.ion()
fig = plt.figure()
ax = fig.add_subplot(111)

im = ax.imshow(U+V, vmin = 0, vmax = 1)
plt.colorbar(im,ax=ax)


for x in range(150,300):#USER INFO: ADJUST THE RANGE OF FRAMES TO BE DISPLAYED HERE
    U = a[x][:,:,0]
    V = a[x][:,:,1]
    ax.cla()
    im = ax.imshow(U+V, vmin = 0, vmax = 1, cmap = "gray")
    fig.canvas.draw()

# Multiframe direction

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import cv2
import imageio

width = 120
height = 67

matplotlib.rcParams['animation.embed_limit'] = 2**128


video_file = os.getcwd() + "/trailer.mp4"#USER INFO: ADJUST THE ORIGINAL VIDEO PATH HERE

cap = cv2.VideoCapture(video_file)
video_reader = imageio.get_reader(video_file)

plt.ioff()

start_frame = 1000#USER INFO: ADJUST START OF THE FRAMES TO BE ANIMATED HERE
frames = 50#USER INFO: ADJUST THE RANGE OF FRAMES TO BE DISPLAYED HERE
skip_factor = 1#speed up playback by skipping frames?
rescaled = a[0]


X, Y = np.meshgrid(np.arange(width), np.arange(height))


fig, ax = plt.subplots(1,1, figsize = (16,9))
Q = ax.quiver(X, Y, np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]), 
              units='xy', color='g', scale = 1.0)
ax.imshow(cap.read()[1])

def animate(i):
    print(".", end ="")
    i *= skip_factor#skip frames
    
    img = None
    if(False):
        cap.set(1, start_frame + i)
        ret, img = cap.read()
    else:
        img = video_reader.get_data(start_frame+ i)
    ax.imshow(cv2.resize(img, (256,144)), extent = [0,width,0,height], origin = "upper")
    
    rescaled = np.array(a[start_frame+i], dtype=np.float32)
    rescaled = rescaled / 16.0 #Get displacement in pixels of the output not input
    rescaled = cv2.resize(rescaled, (width,height)) 
    
    
    Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frames)

from IPython.display import HTML
HTML(ani.to_jshtml())

# Some postprocesing (Median filter)

In [ ]:
np.set_printoptions(5)

In [ ]:
a = a[1000:1000+300,:,:,:]#1000 adjust to have a different part filtered

In [ ]:
a.shape

In [ ]:
from scipy.ndimage import median_filter

In [ ]:
x_comp = a[:,:,:,0]
y_comp = a[:,:,:,1]

In [ ]:
a1 = np.ndarray((300,67,120,2))

In [ ]:
x_comp = median_filter(x_comp, size = 2)

In [ ]:
y_comp = median_filter(y_comp, size = 2)

In [ ]:
a[:,:,:,0] = x_comp#overwrite old a
a[:,:,:,1] = y_comp

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import cv2
import imageio

width = 120
height = 67

matplotlib.rcParams['animation.embed_limit'] = 2**128


video_file = os.getcwd() + "/trailer.mp4"#USER INFO: ADJUST THE ORIGINAL VIDEO PATH HERE

cap = cv2.VideoCapture(video_file)
video_reader = imageio.get_reader(video_file)

plt.ioff()

start_frame = 0#USER INFO: ADJUST START OF THE FRAMES TO BE ANIMATED HERE
frames = 50#USER INFO: ADJUST THE RANGE OF FRAMES TO BE DISPLAYED HERE
skip_factor = 1#speed up playback by skipping frames?
rescaled = a[0]


X, Y = np.meshgrid(np.arange(width), np.arange(height))


fig, ax = plt.subplots(1,1, figsize = (16,9))
Q = ax.quiver(X, Y, np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]), 
              units='xy', color='g', scale = 1.0)
ax.imshow(cap.read()[1])

def animate(i):
    print(".", end ="")
    i *= skip_factor#skip frames
    
    img = None
    if(False):
        cap.set(1, start_frame + i)
        ret, img = cap.read()
    else:
        img = video_reader.get_data(1000+start_frame+ i)#1000!!!!!!! adjust if having a different part filtered
    ax.imshow(cv2.resize(img, (256,144)), extent = [0,width,0,height], origin = "upper")
    
    rescaled = np.array(a[start_frame+i], dtype=np.float32)
    rescaled = rescaled / 16.0 #Get displacement in pixels of the output not input
    rescaled = cv2.resize(rescaled, (width,height)) 
    
    
    Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frames)

from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
def histogram(vector_array, weighted=True):
    """ Computes weighted histograms
    Args:
        vector_array: Array of shape width x height x 2 representing motion vectors for each pixel
        weigthed: Boolean that states wheather bins of directions of motion are to be weighted by the length of
                  the vectors summerized in
    Returns: Weighted Histogram with 36 bins (), bin edges, 
            2d array of directions in degree (upwards, rightwards, downwards, leftwards),
            2d array of lengths of vectors """
    
    ary = np.array([vector_array[:,:,0].flatten(),vector_array[:,:,1].flatten()])
    deg = np.degrees(np.arctan2(ary[0],ary[1]))+180
    length = np.linalg.norm(ary, axis = 0)#.reshape(d[0].shape[0:2])
    bins = (np.arange(36 +1)*10) #start at 0 go up to 360. Mind the rightmost edge has to be defined, hence +1.
    
    hist = None
    bin_edges = None
    if weighted:
        hist, bin_edges = np.histogram(deg, weights = length, bins = bins)
    else:
        hist, bin_edges = np.histogram(deg, bins = bins)
    
    return hist, bin_edges, deg.reshape((vector_array.shape[0], vector_array.shape[1])), length.reshape((vector_array.shape[0], vector_array.shape[1]))

def histograms(motion_tensor, weighted=True, frame_range = None):
    """ Computes histograms for each frame of motion vectors and returns it as a 2d array
    Args:
        motion_tensor: Tensor of shape n_frames x width x height x 2 representing motion vectors (dx and dy)
        weigthed: Boolean that states wheather bins of directions of motion are to be weighted by the length of
                  the vectors summerized in
        frame_range: List of two ints that states the beginning and the end of the sequence of frames to be processed
    """
    data = motion_tensor
    if frame_range:
        data = data[frame_range[0]:frame_range[1]]
        
    outary = np.ndarray((data.shape[0],36))
        
    for d, i in zip(data, range(data.shape[0])):
        outary[i] = histogram(d)[0]
        if i%100 == 0:
            print(i/data.shape[0])
        
    return outary.T

In [ ]:
mothistmap = histogram(a[0])

In [ ]:
mothistmap = histograms(a, False, [0,300])

In [ ]:
%matplotlib inline
fig = plt.subplots(figsize=(15,15))
plt.imshow(mothistmap)